In [113]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
from dash.dash_table import DataTable
from datetime import date


In [114]:
APP_STYLE = {
    'font-family': 'Arial, sans-serif',
    'margin': '0 auto',
    'padding': '20px',
    'max-width': '800px',
    'background-color': '#ffffff',  # Softer white background
    'box-shadow': '0 4px 8px rgba(0, 0, 0, 0.1)',
    'border-radius': '10px',
    'padding-bottom': '30px',
}

HEADER_STYLE = {
    'text-align': 'center',
    'padding': '15px 0',
    'background-color': '#0066cc',  # Calming blue header
    'color': 'white',
    'border-radius': '10px 10px 0 0',
    'font-size': '24px',
    'font-weight': 'bold'
}

NAV_BAR_STYLE = {
    'text-align': 'center',
    'margin': '15px 0',
}

NAV_LINK_STYLE = {
    'margin': '0 10px',
    'padding': '10px 20px',
    'background-color': '#0066cc',  # Matches header
    'color': 'white',
    'border-radius': '5px',
    'text-decoration': 'none',
    'font-weight': 'bold',
    'box-shadow': '0 2px 4px rgba(0, 0, 0, 0.2)',  # Adds depth
}

SECTION_STYLE = {
    'padding': '15px 20px',
    'display': 'flex',
    'flex-direction': 'column',
    'align-items': 'center',
}

BUTTON_STYLE = {
    'background-color': '#0066cc',  # Matches header and nav
    'color': 'white',
    'border': 'none',
    'padding': '10px 30px',
    'border-radius': '5px',
    'cursor': 'pointer',
    'font-size': '16px',
    'margin-top': '15px',
    'box-shadow': '0 2px 4px rgba(0, 0, 0, 0.2)',  # Subtle shadow for depth
}

INPUT_STYLE = {
    'margin': '10px 0',
    'padding': '12px 15px',
    'width': '100%',
    'border': '1px solid #ccc',
    'border-radius': '5px',
    'font-size': '14px',
    'box-shadow': 'inset 0 1px 3px rgba(0, 0, 0, 0.1)',  # Adds dimension
}

DROPDOWN_STYLE = {
    'margin': '10px 0',
    'padding': '12px 15px',
    'width': '100%',
    'height': '50%',
    'border': '1px solid #ccc',
    'border-radius': '1px',
    'font-size': '14px',
    'background-color': '#f9f9f9',  # Softer background to match input
    'box-shadow': 'inset 0 1px 3px rgba(0, 0, 0, 0.1)',  # Same as input
}

DATE_STYLE = {
    'display': 'flex',
    'justify-content': 'space-between',
    'gap': '15px',  # Balanced spacing between date fields
    'width': '100%',
}

DATE_INPUT_STYLE = {
    'padding': '12px 15px',
    'flex': '1',  # Ensures even width for both date inputs
    'border': '1px solid #ccc',
    'border-radius': '5px',
    'font-size': '14px',
    'box-shadow': 'inset 0 1px 3px rgba(0, 0, 0, 0.1)',
}


In [115]:
# Page 1: Student Details Layout
def student_details_layout():
    return html.Div([
        html.H1("Student Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(
                id='admission_number', 
                type='number', 
                placeholder='Admission Number', 
                style=INPUT_STYLE
            ),
            dcc.Input(
                id='aadhar_number', 
                type='number', 
                placeholder='Aadhar Number', 
                style=INPUT_STYLE
            ),
            dcc.Input(
                id='StudentName', 
                type='text', 
                placeholder='Student Name', 
                style=INPUT_STYLE
            ),
            dcc.Input(
                id='FatherName', 
                type='text', 
                placeholder="Father's Name", 
                style=INPUT_STYLE
            ),
            dcc.Dropdown(
                id='gender',
                options=[
                    {'label': 'Boy', 'value': 'Boy'},
                    {'label': 'Girl', 'value': 'Girl'}
                ],
                placeholder="Gender",
                style=DROPDOWN_STYLE
            ),
            html.Div([
                dcc.DatePickerSingle(
                    id='dob', 
                    display_format='DD/MM/YYYY',
                    placeholder="Date of Birth",
                    style=DATE_INPUT_STYLE
                ),
                dcc.DatePickerSingle(
                    id='doj', 
                    display_format='DD/MM/YYYY',
                    placeholder="Date of Joining",
                    style=DATE_INPUT_STYLE
                )
            ], style=DATE_STYLE),
            html.Button(
                'Submit', 
                id='student-details-submit-button', 
                n_clicks=0, 
                style=BUTTON_STYLE
            ),
        ], style=SECTION_STYLE),
        html.A(
            "Back to Home", 
            href='/', 
            style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'}
        )
    ], style=APP_STYLE)


In [116]:
# Page 2: Student Class Details Layout
def student_class_details_layout():
    return html.Div([
        html.H1("Student Class Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='aadhar_number', type='number', placeholder='Aadhar Number', style=INPUT_STYLE),
            dcc.Dropdown(
                id='class', 
                options=[{'label': f'Class {i}', 'value': i} for i in range(1, 13)],
                placeholder="Class",
                style=DROPDOWN_STYLE
            ),
            dcc.Input(id='section', type='text', placeholder='Section', style=INPUT_STYLE),
            dcc.Input(id='contact_number', type='tel', placeholder='Contact Number', style=INPUT_STYLE),
            dcc.Input(id='village', type='text', placeholder='Village', style=INPUT_STYLE),
            dcc.Input(id='current_year', type='number', placeholder='Year', style=INPUT_STYLE),
            dcc.Dropdown(id='enrolled', options=[
                {'label': 'Yes', 'value': 'Yes'},
                {'label': 'No', 'value': 'No'}
            ], placeholder="Currently Enrolled", style=DROPDOWN_STYLE),
            html.Button('Submit', id='student-class-submit-button', n_clicks=0, style=BUTTON_STYLE),
        ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [117]:
# Page 3: Fee Details Layout
def fee_details_layout():
    return html.Div([
        html.H1("Fee Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='aadhar_number', type='number', placeholder='Aadhar Number', style=INPUT_STYLE),
            dcc.Input(id='year', type='number', placeholder='Year', style=INPUT_STYLE),
            dcc.Input(id='school_fee', type='number', placeholder='School Fee', style=INPUT_STYLE),
            dcc.Dropdown(
                id='school_fee_concession_reason',
                options=[
                    {'label': reason, 'value': reason} for reason in [
                        'Staff', 'Sibling', 'OTP', 'TF', 'FP', 'EC', 'SC', 'General'
                    ]
                ],
                placeholder="School Fee Concession Reason",
                style=DROPDOWN_STYLE
            ),
            dcc.Dropdown(
                id='transport_used',
                options=[
                    {'label': 'Yes', 'value': 'Yes'},
                    {'label': 'No', 'value': 'No'}
                ],
                placeholder="Transport Used",
                style=DROPDOWN_STYLE
            ),
            dcc.Input(id='pickup_point', type='text', placeholder='Pick-Up Point', style=INPUT_STYLE, disabled=True),
            dcc.Input(id='transport_route', type='number', placeholder='Route Number', style=INPUT_STYLE, disabled=True),
            dcc.Input(id='transport_fee', type='number', placeholder='Transport Fee', style=INPUT_STYLE, disabled=True),
            dcc.Input(id='transport_fee_concession', type='text', placeholder='Transport Fee Concession', style=INPUT_STYLE, disabled=True),
            dcc.Input(id='application_fee', type='number', placeholder='Application Fee', style=INPUT_STYLE),
            html.Button('Submit', id='fee-details-submit-button', n_clicks=0, style=BUTTON_STYLE),
            ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [118]:
# Page 4: Fee Payment Details Layout
def fee_type_selection_layout():
    return html.Div([
        html.H1("Fee Payment Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='aadhar_number', type='number', placeholder='Aadhar Number', style=INPUT_STYLE),
            dcc.Input(id='year', type='number', placeholder='Year', style=INPUT_STYLE),
            dcc.Dropdown(
                id='fee_type',
                options=[
                    {'label': 'School Fee - Term 1', 'value': 'school_fee_term_1'},
                    {'label': 'School Fee - Term 2', 'value': 'school_fee_term_2'},
                    {'label': 'School Fee - Term 3', 'value': 'school_fee_term_3'},
                    {'label': 'Transport Fee - Term 1', 'value': 'transport_fee_term_1'},
                    {'label': 'Transport Fee - Term 2', 'value': 'transport_fee_term_2'},
                    {'label': 'Transport Fee - Term 3', 'value': 'transport_fee_term_3'},
                    {'label': 'Application Fee', 'value': 'application_fee'}
                ],
                value=None,  # Default to None
                placeholder="Select Fee Type",
                style=DROPDOWN_STYLE
            ),
            html.Button('Next', id='next-button', n_clicks=0, style=BUTTON_STYLE),
            html.Div(id='fee-content'),
        ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [119]:
# Page 4.1: School Fee Payment Details Layout
def school_fee_payment_layout():
    return html.Div([
        html.H1("School Fee Payment Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='1st_term_paid', type='number', placeholder='1st Term School Fee Paid', style=INPUT_STYLE),
            dcc.Input(id='1st_term_due', type='number', placeholder='1st Term School Fee Due', style=INPUT_STYLE),
            dcc.Input(id='1st_term_receipt', type='number', placeholder='1st Term School Fee Receipt Number', style=INPUT_STYLE),
            dcc.DatePickerSingle(
                id='1st_term_paid_date',
                display_format='DD/MM/YYYY',
                placeholder="School Fee - I Term Date Paid",
                date=date.today(),  # Set the default date to today
                style={'margin-top': '10px'}
            ),
            html.Button('Submit', id='school-fee-submit-button', n_clicks=0, style=BUTTON_STYLE),
            ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [120]:
# Page 4.2: Transport Fee Payment Details Layout
def transport_fee_payment_layout():
    return html.Div([
        html.H1("Transport Fee Payment Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='1st_term_paid', type='number', placeholder='1st Term Transport Fee Paid', style=INPUT_STYLE),
            dcc.Input(id='1st_term_due', type='number', placeholder='1st Term Transport Fee Due', style=INPUT_STYLE),
            dcc.Input(id='1st_term_receipt', type='number', placeholder='1st Term Transport Fee Receipt Number', style=INPUT_STYLE),
            dcc.DatePickerSingle(
                id='1st_term_paid_date',
                display_format='DD/MM/YYYY',
                placeholder="Transport Fee - I Term Date Paid",
                date=date.today(),  # Set the default date to today
                style={'margin-top': '10px'}
            ),
            html.Button('Submit', id='transport-fee-submit-button', n_clicks=0, style=BUTTON_STYLE),
            ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [121]:
# Page 4.3: Application Fee Payment Details Layout
def application_fee_payment_layout():
    return html.Div([
        html.H1("Application Fee Payment Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='application_paid', type='number', placeholder='1st Term Application Fee Paid', style=INPUT_STYLE),
            dcc.Input(id='application_due', type='number', placeholder='1st Term Application Fee Due', style=INPUT_STYLE),
            dcc.Input(id='application_receipt', type='number', placeholder='1st Term Application Fee Receipt Number', style=INPUT_STYLE),
            dcc.DatePickerSingle(
                id='application_paid_date',
                display_format='DD/MM/YYYY',
                placeholder="Application Fee - Date Paid",
                date=date.today(),  # Set the default date to today
                style={'margin-top': '10px'}
            ),
            html.Button('Submit', id='application-fee-submit-button', n_clicks=0, style=BUTTON_STYLE),
            ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [122]:
# Page 5: View Records Layout
def view_records_layout():
    return html.Div([
        html.H1("View Records", style=HEADER_STYLE),
        html.Div([
            dcc.Dropdown(
                id='view-class', 
                options=[{'label': f'Class {i}', 'value': i} for i in range(1, 13)],
                placeholder="Select Class",
                style=DROPDOWN_STYLE
            ),
            dcc.Dropdown(
                id='view-year', 
                options=[{'label': str(year), 'value': year} for year in range(2020, 2025)],
                placeholder="Select Year",
                style=DROPDOWN_STYLE
            ),
            DataTable(
                id='student-table',
                columns=[
                    {'name': 'Admission Number', 'id': 'admission_number'},
                    {'name': 'Student Name', 'id': 'student_name'},
                    {'name': 'Class', 'id': 'class'},
                    {'name': 'Section', 'id': 'section'}
                ],
                style_table={'margin-top': '20px'}
            )
        ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [123]:
# Define app with URL routing
app = dash.Dash(__name__, suppress_callback_exceptions=True)

# Create navigation bar
def navigation_bar():
    return html.Div([
        html.A("Student Details", href='/student-details', style=NAV_LINK_STYLE),
        html.A("Student Class Details", href='/student-class-details', style=NAV_LINK_STYLE),
        html.A("Fee Details", href='/fee-details', style=NAV_LINK_STYLE),
        html.A("Fee Payment Details", href='/fee-payment-details', style=NAV_LINK_STYLE),
        html.A("View Records", href='/view-records', style=NAV_LINK_STYLE),
    ], style=NAV_BAR_STYLE)

# Set app layout with navigation bar and page content
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),  # Tracks current URL
    navigation_bar(),
    html.Div(id='page-content')
])

# Update page content based on URL
@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)

def display_page(pathname):
    if pathname == '/student-details':
        return student_details_layout()
    elif pathname == '/student-class-details':
        return student_class_details_layout()
    elif pathname == '/fee-details':
        return fee_details_layout()
    elif pathname == '/view-records':
        return view_records_layout()
    elif pathname == '/fee-payment-details':
        return fee_type_selection_layout()
    elif pathname == '/':
        return html.Div([
            html.H1("Welcome to the Student Management System", style=HEADER_STYLE),
            html.P("Use the navigation bar above to explore different pages.", style={'text-align': 'center'})
        ], style=APP_STYLE)
    else:
        return html.Div([
            html.H1('404: Page not found', style=HEADER_STYLE),
            html.P('The page you are looking for does not exist.', style={'text-align': 'center'})
        ], style=APP_STYLE)

In [124]:
# Define callback
@app.callback(
    Output('fee-content', 'children'),
    [Input('next-button', 'n_clicks')],
    [State('fee_type', 'value')]
)
def navigate_to_fee_section(n_clicks, fee_type):
    if n_clicks > 0:
        if fee_type in ['school_fee_term_1', 'school_fee_term_2', 'school_fee_term_3']:
            return school_fee_payment_layout()
        elif fee_type in ['transport_fee_term_1', 'transport_fee_term_2', 'transport_fee_term_3']:
            return transport_fee_payment_layout()
        elif fee_type == 'application_fee':
            return application_fee_payment_layout()
        else:
            return html.Div("Please select a valid fee type.")
    return html.Div()  # Default empty content

In [125]:
# Toggle transport inputs based on transport usage selection
@app.callback(
    [
        Output('pickup_point', 'disabled'),
        Output('transport_route', 'disabled'),
        Output('transport_fee', 'disabled'),
        Output('transport_fee_concession', 'disabled'),
    ],
    Input('transport_used', 'value')
)
def toggle_transport_inputs(transport_used):
    if transport_used == 'No':
        return True, True, True, True  # Disable fields
    return False, False, False, False  # Enable fields

In [126]:
if __name__ == '__main__':
    app.run_server(debug=True)